In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
import numpy as np
import random
import time

#We import some packages.

In [ ]:
#We import the data.

VectorizedData= np.load("SafetyWeights.npy")
Labels= np.load("SafetyLabels.npy")


rebalanceSamples=True
if(rebalanceSamples):
    locations=[]
    fullList=np.zeros(1,dtype=np.int)
    shortest= 99999999999999999999;
    for k in range(Labels.shape[1]):
        locations.append(np.where(Labels[:,k]==1)[0])
        shortest=min(shortest,sum(Labels[:,k]==1))
    for k in range(Labels.shape[1]):
            fullList=  np.concatenate([fullList,np.random.choice(locations[k], shortest,replace=False)] )
    fullList=fullList[1:]
    VectorizedData=VectorizedData[fullList]
    Labels=Labels[fullList]

    #We shuffle and cut the data
myPerm= np.random.permutation(VectorizedData.shape[0])
trainSize= round(0.8*VectorizedData.shape[0])
VectorizedData=VectorizedData[myPerm]
Labels=Labels[myPerm]

#Note: If some sort of under or over sampling was to be
#done in order to correct for imbalanced label sets
#this is probably the place to do it.
trainData= VectorizedData[:trainSize,:]
trainLabel= Labels[:trainSize,:]
testData= VectorizedData[trainSize:,:]
testLabel= Labels[trainSize:,:]

In [ ]:
#We create a model.

#Please note that the model give here is completely Ad Hoc.
#There are (hopefully) other models that work better.
newModel = Sequential()
newModel.add(Dense(256, input_dim=trainData.shape[1], activation='relu'))
newModel.add(Dense(256, activation='relu'))
newModel.add(Dense(testLabel.shape[1], activation='softmax'))

newModel.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
startTrainTime = time.time()
newModel.fit(trainData,trainLabel, epochs=15, batch_size=128)
endTime = time.time()
print('\n Time to train = {} s.'.format(endTime - startTrainTime))

In [ ]:
y_pred= newModel.predict(testData)[:,1]
y_true = testLabel[:,1]

y_pred_pos= np.round(y_pred)
y_pred_neg= 1-y_pred_pos

tp = sum(y_true * y_pred_pos)
tn = sum((1-y_true) * y_pred_neg)

fp = sum((1-y_true) * y_pred_pos)
fn = sum (y_true * y_pred_neg)

precision = tp/(tp+fp)
recall= tp/(tp+fn)

print('Precision:')
print(precision)
print('\n Recall:')
print(recall)

print('\n Accuracy:')
print((tp+tn)/(tp+tn+fp+fn))